In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from math import log10

# For reproducibility
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cpu


In [2]:
class StrongLensingDataset(Dataset):
    def __init__(self, lr_files, hr_files):
        self.lr_files = lr_files
        self.hr_files = hr_files
        
    def __len__(self):
        return len(self.lr_files)
    
    def __getitem__(self, idx):
        lr_path = self.lr_files[idx]
        hr_path = self.hr_files[idx]
        
        # Load images (expected shape: (H, W) or (C, H, W))
        lr_img = np.load(lr_path)
        hr_img = np.load(hr_path)
        
        # If single-channel images, add channel dimension (result: (1, H, W))
        if lr_img.ndim == 2:
            lr_img = np.expand_dims(lr_img, axis=0)
        if hr_img.ndim == 2:
            hr_img = np.expand_dims(hr_img, axis=0)
        
        # Convert to float32 and normalize to [0, 1]
        lr_img = lr_img.astype(np.float32) / (lr_img.max() if lr_img.max() != 0 else 1)
        hr_img = hr_img.astype(np.float32) / (hr_img.max() if hr_img.max() != 0 else 1)
        
        # Convert to torch tensors
        lr_tensor = torch.from_numpy(lr_img)
        hr_tensor = torch.from_numpy(hr_img)
        
        return lr_tensor, hr_tensor


In [4]:
# Update these paths to your dataset directories.
lr_folder = "/Users/EndUser/Downloads/dataset-3/LR"  # Adjust folder name as needed
hr_folder = "/Users/EndUser/Downloads/dataset-3/HR"

# List and sort .npy files in each folder
lr_files = sorted([os.path.join(lr_folder, f) for f in os.listdir(lr_folder) if f.endswith('.npy')])
hr_files = sorted([os.path.join(hr_folder, f) for f in os.listdir(hr_folder) if f.endswith('.npy')])

assert len(lr_files) == len(hr_files), "Mismatch: Number of LR and HR files do not match."

# Split into training and validation sets (90:10 split)
train_lr, val_lr, train_hr, val_hr = train_test_split(lr_files, hr_files, test_size=0.1, random_state=SEED)

# Create Dataset objects
train_dataset = StrongLensingDataset(train_lr, train_hr)
val_dataset   = StrongLensingDataset(val_lr, val_hr)

# Create DataLoaders
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

print(f"Training samples: {len(train_dataset)}, Validation samples: {len(val_dataset)}")


Training samples: 270, Validation samples: 30


In [5]:
class SRCNN(nn.Module):
    def __init__(self, num_channels=1):
        super(SRCNN, self).__init__()
        # Three convolutional layers as in the SRCNN paper
        self.conv1 = nn.Conv2d(num_channels, 64, kernel_size=9, padding=4)
        self.conv2 = nn.Conv2d(64, 32, kernel_size=1, padding=0)
        self.conv3 = nn.Conv2d(32, num_channels, kernel_size=5, padding=2)
        self.relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        # x is expected to be [B, 1, 128, 128]
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        # Output: [B, 1, 128, 128]
        return x

model = SRCNN(num_channels=1).to(device)


In [6]:
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 10


In [7]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for lr_imgs, hr_imgs in train_loader:
        # lr_imgs: [B, 1, 64, 64]; hr_imgs: [B, 1, 128, 128]
        lr_imgs = lr_imgs.to(device)
        hr_imgs = hr_imgs.to(device)
        
        # Upsample LR images to 128×128 using bicubic interpolation
        lr_imgs_upsampled = F.interpolate(lr_imgs, size=(128, 128), mode='bicubic', align_corners=False)
        # Now lr_imgs_upsampled: [B, 1, 128, 128]
        
        optimizer.zero_grad()
        sr_imgs = model(lr_imgs_upsampled)  # Output: [B, 1, 128, 128]
        
        loss = criterion(sr_imgs, hr_imgs)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * lr_imgs.size(0)
    
    epoch_loss = running_loss / len(train_dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.6f}")


Epoch [1/10], Train Loss: 0.003020
Epoch [2/10], Train Loss: 0.001764
Epoch [3/10], Train Loss: 0.001602
Epoch [4/10], Train Loss: 0.001562
Epoch [5/10], Train Loss: 0.001519
Epoch [6/10], Train Loss: 0.001498
Epoch [7/10], Train Loss: 0.001483
Epoch [8/10], Train Loss: 0.001467
Epoch [9/10], Train Loss: 0.001437
Epoch [10/10], Train Loss: 0.001434


In [8]:
model.eval()
val_loss = 0.0

with torch.no_grad():
    for lr_imgs, hr_imgs in val_loader:
        lr_imgs = lr_imgs.to(device)
        hr_imgs = hr_imgs.to(device)
        
        # Upsample LR images to 128×128
        lr_imgs_upsampled = F.interpolate(lr_imgs, size=(128, 128), mode='bicubic', align_corners=False)
        sr_imgs = model(lr_imgs_upsampled)
        
        loss = criterion(sr_imgs, hr_imgs)
        val_loss += loss.item() * lr_imgs.size(0)

val_loss /= len(val_dataset)
print(f"Validation Loss: {val_loss:.6f}")


Validation Loss: 0.000878
